In [1]:
# Import packages

import numpy as np
import pandas as pd
import random
import math


In [2]:
# Function for importing internal datasets

def get_internal_data():
    gpt2_df = pd.read_csv('source/gpt2_survey.csv', index_col = 0)
    t5_df = pd.read_csv('source/t5_survey.csv', index_col = 0)
    
    # Get subset
    t5_df = t5_df[['original','model_2']]
    t5_df['source'] = gpt2_df[['source']]# np.random.choice(['cs_conf','student','medium'], t5_df.shape[0])
    gpt2_df = gpt2_df[['sent','output','source']]
    
    gpt2_df.columns = ['sent1','sent2','source']
    t5_df.columns = ['sent1','sent2','source']
    
    gpt2_df['model'] = 'gpt2'
    t5_df['model'] = 't5'

    gpt2_df['og'] = 1
    t5_df['og'] = 1

    gpt2_df['sent1'] = gpt2_df['sent1'].str.lower().str.capitalize()
    gpt2_df['sent2'] = gpt2_df['sent2'].str.lower().str.capitalize()
    t5_df['sent1'] = t5_df['sent1'].str.lower().str.capitalize()
    t5_df['sent2'] = t5_df['sent2'].str.lower().str.capitalize()
    
    subset1 = [*range(0, 9), *range(33, 42), *range(81, 90)]
    subset2 = [*range(9, 18), *range(42, 51), *range(90, 99)]
    
    group1_df = gpt2_df.iloc[subset1,:].append(t5_df.iloc[subset2,:], ignore_index=True)
    group1_df['qid'] = group1_df.index
    group2_df = t5_df.iloc[subset1,:].append(gpt2_df.iloc[subset2,:], ignore_index=True)
    group2_df['qid'] = group2_df.index
    
    return group1_df, group2_df


In [3]:
# Function for importing external datasets

def get_external_data():

    sent_sample = pd.read_csv('source/qualtrics_external.csv', index_col = 0)
    t5_paraphrases = pd.read_csv('source/qualtrics_t5_external.csv', index_col = 0)

    sent_sample = pd.read_csv('source/qualtrics_external.csv')
    t5_paraphrases = pd.read_csv('source/qualtrics_t5_external.csv')
    sentence_pairs_df = sent_sample.merge(t5_paraphrases, left_index = True, right_index = True)

    sentence_pairs_df = sentence_pairs_df[['original','paraphrase','source']]
    
    sentence_pairs_df.columns = ['sent1','sent2','source']
    
    sentence_pairs_df['model'] = 't5'
    
    sentence_pairs_df['og'] = 1
    
    sentence_pairs_df['sent1'] = sentence_pairs_df['sent1'].str.lower().str.capitalize()
    sentence_pairs_df['sent2'] = sentence_pairs_df['sent2'].str.lower().str.capitalize()
    
    sentence_pairs_df['qid'] = sentence_pairs_df.index

    return sentence_pairs_df


In [4]:
# Function for adding randomized repeating sentences

def create_intrarater(sentence_pairs_df, repeating):

    sentence_pairs = sentence_pairs_df.values.tolist()

    intrarater_pairs = sentence_pairs + random.sample(sentence_pairs, repeating)

    # random.shuffle(intrarater_pairs)

    intrarater_pairs_df = pd.DataFrame(intrarater_pairs, columns = sentence_pairs_df.columns)

    return intrarater_pairs_df


In [5]:
# Function for pseudo-randomizing order of choices for internal evaluation

def randomize_order(intrarater_pairs_df):
    
    random_pairs = []
    
    intrarater_pairs = intrarater_pairs_df.values.tolist()
    
    for i in intrarater_pairs:
        sent1 = i[0]
        sent2 = i[1]
        source = i[2]
        model = i[3]
        og = i[4]
        qid = i[5]
        
        if random.random() < 0.5:
            random_pair = [sent2, sent1, source, model, '2', qid]
        else:
            random_pair = [sent1, sent2, source, model, '1', qid]
        
        random_pairs.append(random_pair)

    random_order_df = pd.DataFrame(random_pairs, columns = intrarater_pairs_df.columns)

    return random_order_df


In [6]:
# Function for creating import txt for Qualtrics

def qualtrics_txt(df, txt_name, repeat_block = False):

    with open(txt_name,'w') as f:
        f.write('[[AdvancedFormat]]\n\n[[Block:MC Block]]\n\n')
        
        for i in range(df.shape[0]):
            row = df.iloc[i,:]
            sent1 = row['sent1']
            sent2 = row['sent2']
            if repeat_block != False and i == df.shape[0]-repeat_block:
                newblock = "[[Block:Repeat Block]]\n\n"
                f.write(newblock)
            question = "[[Question:MC:SingleAnswer:Vertical]]\nWhich sentence sound more scientific? (Please try to disregard semantics and focus only on the style.)\n[[Choices]]"
            question = question+"\n"+sent1+"\n"+sent2+"\n\n"
            f.write(question)


In [7]:
# Create dataset for internal evaluation

group1_df, group2_df = get_internal_data()

group1_df = randomize_order(create_intrarater(group1_df, repeating = 5))
group2_df = randomize_order(create_intrarater(group2_df, repeating = 5))

group1_df.to_csv('output/group1_df.csv')
group2_df.to_csv('output/group2_df.csv')

qualtrics_txt(group1_df, txt_name = 'output/qualtrics_group1.txt', repeat_block = False)
qualtrics_txt(group2_df, txt_name = 'output/qualtrics_group2.txt', repeat_block = False)


In [8]:
# Create dataset for external evaluation

sentence_pairs_df = get_external_data()

sentence_pairs_df = create_intrarater(sentence_pairs_df, repeating = 5)

sentence_pairs_df.to_csv('output/sentence_pairs_df.csv')

qualtrics_txt(sentence_pairs_df, txt_name = 'output/qualtrics_external.txt', repeat_block = 5)
